In [23]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import re

In [24]:
df = pd.read_csv('../website_classification.csv')
df

,website_url,cleaned_website_text,Category
0,https://www.fubo.tv/lp/StreamTV/?irad=343747&i...,stream live tv sports fubotv free trial sport ...,Streaming Services
1,https://www.hulu.com/,stream tv movie live online hulu watch tv show...,Streaming Services
2,https://tv.youtube.com/,youtube tv watch dvr live sports shows news st...,Streaming Services
3,https://www.attwatchtv.com/,watchtv stream channel live tv want watchtv ex...,Streaming Services
4,https://www.klowdtv.com/,live tv streaming klowdtv klowdtv provide live...,Streaming Services
...,...,...,...
1071,https://www.solarwinds.com/security,it security management tools free software tri...,Cyber Security
1072,https://www.axonius.com,axonius cybersecurity asset management saas ma...,Cyber Security
1073,https://www.secureworks.com,secureworks cybersecurity leader proven threat...,Cyber Security
1074,https://www.trustwave.com,leading managed detection and response trustwa...,Cyber Security


In [25]:
class ScrapTool:
    def visit_url(self,website_url):
        '''
        Visit URL. Download the Content. Initialize the beautifulsoup object. Call parsing methods. Return Series object.
        '''
        content = requests.get(website_url,timeout=60).content
        soup = BeautifulSoup(content,'lxml')
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "website_text": self.get_html_title_tag(soup)+self.get_html_meta_tags(soup)+self.get_html_heading_tags(soup)+
                                                        self.get_text_content(soup)
        }
        
        #Convert to Series object and return
        return pd.Series(result)
    
    def get_website_name(self,website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])
    
    def get_html_title_tag(self,soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)
    
    def get_html_meta_tags(self,soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name=="meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords','description']]
        return ' '.join(content)
    
    def get_html_heading_tags(self,soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1","h2","h3","h4","h5","h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)
    
    def get_text_content(self,soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]',"h1","h2","h3","h4","h5","h6","noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore\
                and isinstance(tag, bs4.element.Comment)==False\
                and not stripped_tag.isnumeric()\
                and len(stripped_tag)>0:
                result.append(stripped_tag)
        return ' '.join(result)

In [26]:
def cleaning_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r' +',' ',text)
    return text

In [27]:
def content_generation(website):
    scrapTool = ScrapTool()
    try:
        web = dict(scrapTool.visit_url(website))
        text = cleaning_text(web['website_text'])
        return text
    except:
        print('Error: ',website)
        return None

In [7]:
websites = [
    "https://www.cyberbit.com/",
    "https://www.sonicwall.com/"
]


for website in websites:
    text = content_generation(website)
    if text is not None:
        web = [website,text,'Cyber Security']
        df.loc[len(df)] = web

Error:  https://www.cyberbit.com/
Error:  https://www.sonicwall.com/


In [28]:
df['Category'].value_counts()

Business/Corporate                 109
Streaming Services                 104
Sports                             104
E-Commerce                         102
Games                               98
News                                95
Computers and Technology            93
Cyber Security                      91
Law and Government                  84
Cryptocurrency                      80
Social Networking and Messaging     79
Adult                               21
Forums                              16
Name: Category, dtype: int64

In [29]:
df2 = df[(df["Category"] == "Cryptocurrency") |( df["Category"]=="Computers and Technology")|( df["Category"]=="Cyber Security")]

In [30]:
df2

,website_url,cleaned_website_text,Category
682,http://www.kasrl.org/jaffe.html,japanese female facial expression jaffe datase...,Computers and Technology
683,https://www.gmdhshell.com/,best predictive analytic software free academi...,Computers and Technology
684,http://en.wikipedia.org/wiki/Neural_network,neural network wikipedia neural network conten...,Computers and Technology
685,http://cadingandcoding.blogspot.com/,cading coding cading coding tuesday march auto...,Computers and Technology
686,http://www.panzercad.com/,welcome bluehost rate web host provider free c...,Computers and Technology
...,...,...,...
1071,https://www.solarwinds.com/security,it security management tools free software tri...,Cyber Security
1072,https://www.axonius.com,axonius cybersecurity asset management saas ma...,Cyber Security
1073,https://www.secureworks.com,secureworks cybersecurity leader proven threat...,Cyber Security
1074,https://www.trustwave.com,leading managed detection and response trustwa...,Cyber Security


In [32]:
df["Category"] = df["Category"].apply(lambda x: "Computers and Technology" if (x=="Cryptocurrency" or x=="Cyber Security") else x)

In [33]:
df['Category'].value_counts()

Computers and Technology           264
Business/Corporate                 109
Streaming Services                 104
Sports                             104
E-Commerce                         102
Games                               98
News                                95
Law and Government                  84
Social Networking and Messaging     79
Adult                               21
Forums                              16
Name: Category, dtype: int64

In [34]:
df.reset_index(drop=True,inplace=True)
df.to_csv("website_classification.csv",index=False)